In [1]:
import numpy as np

In [2]:
i = [[2,3,5,1],
     [1,-0.3,0.5,0.8],
     [0.5,2,0.8,-2]]
len(np.array(i).T)

4

In [3]:
np.random.seed(0)
def weightdef(n_shape):
    return np.random.randn(n_shape,n_shape)
    

In [4]:
def layer(i):
    w=weightdef(len(np.array(i).T))
    bias=np.zeros((1,len(np.array(i).T)))
    return ReLU((np.dot(i,w)+bias))

In [5]:
exp_val=np.exp(i-np.max(i,axis=1,keepdims=True)) 
output= exp_val/np.sum(exp_val, axis=1, keepdims=True)

In [6]:
print(exp_val)

[[0.04978707 0.13533528 1.         0.01831564]
 [1.         0.27253179 0.60653066 0.81873075]
 [0.22313016 1.         0.30119421 0.01831564]]


In [7]:
print(output)

[[0.0413707  0.11245721 0.83095266 0.01521943]
 [0.37067333 0.10102027 0.22482474 0.30348166]
 [0.14464176 0.64823938 0.19524595 0.01187292]]


In [8]:
class Layer:
    def __init__(self, num_input, num_neurons):
        self.weight=0.01*np.random.randn(num_input,num_neurons)
        self.bias=np.zeros((1,num_neurons))
    def forward(self,inputs):
         self.output= np.dot(inputs,self.weight)+self.bias

In [9]:
class ReLU:
    def forward(self,x):
        self.output= np.maximum(0,x)

In [10]:
def fix_dim_error(X):
    exp_val=[]
    for i in X:
        tempexp=[]
        for j in i:
            val=j
            tempexp.append(val)
        exp_val.append(tempexp)
    return exp_val

In [11]:
class Softmax:
    def forward(self,x):
        x=fix_dim_error(x)
        exp_val=np.exp(x - np.max(x, axis=1, keepdims=True))
        self.output= (exp_val / np.sum(exp_val, axis=1, keepdims=True))

In [12]:
act=Softmax()
act.forward(i)
print(act.output)
print(np.sum(act.output,axis=1,keepdims=True))

[[0.0413707  0.11245721 0.83095266 0.01521943]
 [0.37067333 0.10102027 0.22482474 0.30348166]
 [0.14464176 0.64823938 0.19524595 0.01187292]]
[[1.]
 [1.]
 [1.]]


In [13]:
class Loss:
    def calculate(self, result, y):
        sample_loss= self.forward(result,y)
        return np.mean(sample_loss)

In [14]:
class Categorical_Crossentropy(Loss):
    def forward(self,predict,y):
        predict_clipped= np.clip(predict,1e-7,1-1e-7)
        if(len(y.shape)==1):
            currect_pred = predict_clipped[range(len(predict_clipped)),y]
        else:
            currect_pred = np.sum(predict_clipped*y, axis=1)
        return -np.log(currect_pred)

In [15]:
def Accuracy(output,target): 
    predict=np.argmax(output, axis=1)
    target=np.argmax(target, axis=1)
    return- np.mean(predict == target)

In [16]:
import pandas as pd

In [17]:
df=pd.read_excel('test1file(fsdnoisy).xlsx')

 

df['feature']=df['feature'].str.lstrip('[')
df['feature']=df['feature'].str.rstrip(']')

features= df['feature'].str.split(expand=True)
classes=df.drop(['feature','Unnamed: 0'],axis=1)

feature_df_test=pd.concat([features,classes],axis=1)
for col in feature_df_test.columns:
             feature_df_test[col] = feature_df_test[col].astype('float', errors='ignore')

In [18]:
X=feature_df_test.values[:,0:-1]
y=np.array(feature_df_test['class'].tolist())


In [19]:
X

array([[-209.93578, 63.42263, -123.69025, ..., 1.4747163, -5.187261,
        2.3625958],
       [-417.291046, 100.148506, -43.8692513, ..., 1.26949859,
        -2.04816842, 0.453536361],
       [-452.77457, 113.24303, -38.501553, ..., 0.72960454, -2.7218251,
        0.81479305],
       ...,
       [-297.74582, 102.95336, -37.630085, ..., 2.3270385, 1.850992,
        -0.24204774],
       [-337.20972, 116.19127, -45.74678, ..., 0.47457361, -1.3576933,
        -1.6199757],
       [-307.73666, 84.129173, -26.636988, ..., 5.0096831, -4.3950777,
        -7.3455243]], dtype=object)

In [20]:
y

array(['dog_bark', 'children_playing', 'children_playing', ...,
       'car_horn', 'car_horn', 'car_horn'], dtype='<U16')

In [21]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [22]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
layer1=Layer(50,8725)
activation1= ReLU()
layer2=Layer(8725,10)
activation_softmax=Softmax()

In [24]:
layer1.forward(X)
activation1.forward(layer1.output)
layer2.forward(activation1.output)
activation_softmax.forward(layer2.output)

In [25]:
loss=Categorical_Crossentropy()
loss.calculate(activation_softmax.output,y)

2.959290435886195

In [26]:
Accuracy(activation_softmax.output,y)

-0.10853868194842407

In [27]:
result=np.argmax(activation_softmax.output, axis=1) == np.argmax(y, axis=1)

In [28]:
np.unique(result)

array([False,  True])

In [29]:
count=0
for i in result:
    if i:
        count+=1
count

947